In [11]:
from deepface import DeepFace
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

from pathlib import Path
home = str(Path.home())
print("HOME_FOLDER is ",home)
print("weight saved at ",home + "\.deepface\weights")

def drawBBoxs(base_img, bboxs, col_dict, colour="blue"):
    output_img = base_img.copy()
    output_img = output_img[:, :, ::-1]

    output_img = Image.fromarray(np.uint8(output_img)).convert('RGB')
    draw = ImageDraw.Draw(output_img)

    for bbox in bboxs:
        try:
            colour = col_dict[bbox[5]]
        except:
            colour = "blue"
        x0 = bbox[0]
        x1 = bbox[0] + bbox[2]
        y0 = bbox[1]
        y1 = bbox[1] + bbox[3]
        draw.rectangle((x0, y0, x1, y1), outline=colour, width=5)
        draw.text((x0 + 10, y0 + 10), bbox[5], fill=colour)
#         draw.text((x0 + 10, y0 + 10), str(round(bbox[4],2)), fill=colour)

        output_img = np.array(output_img)
        output_img = output_img[:, :, ::-1]
        return output_img

HOME_FOLDER is  C:\Users\holeo
weight saved at  C:\Users\holeo\.deepface\weights


In [20]:
#Download Weights
import wget

In [17]:
video_dir = "dataset/video"
video_dir = "NEW_VIDEOS"
output_dir = "dataset/2fps"
videos = os.listdir(video_dir)
for video in videos:
    video_path = video_dir + "/" + video
    output_path = output_dir + "/" + video
    print("ffmpeg -i ", end = "")
    print(video_path, end = " ")
    print("-filter:v fps=2 " , end = "")
    print(output_path)

ffmpeg -i NEW_VIDEOS/mask_on.mp4 -filter:v fps=2 dataset/2fps/mask_on.mp4
ffmpeg -i NEW_VIDEOS/mask_on_simple.mp4 -filter:v fps=2 dataset/2fps/mask_on_simple.mp4
ffmpeg -i NEW_VIDEOS/no_mask_then_got_mask.mp4 -filter:v fps=2 dataset/2fps/no_mask_then_got_mask.mp4
ffmpeg -i NEW_VIDEOS/zlcljp_walking_randomly.mp4 -filter:v fps=2 dataset/2fps/zlcljp_walking_randomly.mp4
ffmpeg -i NEW_VIDEOS/zlcl_crossing_cl_in_front.mp4 -filter:v fps=2 dataset/2fps/zlcl_crossing_cl_in_front.mp4
ffmpeg -i NEW_VIDEOS/zlcl_crossing_zl_in_front.mp4 -filter:v fps=2 dataset/2fps/zlcl_crossing_zl_in_front.mp4
ffmpeg -i NEW_VIDEOS/zl_change_clothes_no_mask.mp4 -filter:v fps=2 dataset/2fps/zl_change_clothes_no_mask.mp4


In [8]:
# Save Face from video
from deepface.commons.functions import initialize_detector, load_image, detect_face2
import numpy as np
import cv2
from PIL import Image, ImageDraw
import os

# video_dir = "dataset/video"
video_dir = "dataset/1fps"
output_dir = "dataset/new/"
videos = os.listdir(video_dir)
print(videos)

for video in videos:
    current_frame = 1
    video_path = video_dir + "/" + video
    fps = 2
    # detector_backend = 'ssd'
    detector_backend = 'mtcnn'
    grayscale = False
    enforce_detection = False
    return_region = False
    db_folder = "dataset"
    
    initialize_detector(detector_backend = detector_backend)
    cap = cv2.VideoCapture(video_path)
    if (cap.isOpened()== False):
        print("Error opening video stream or file")
    ret, frame = cap.read()
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    
    while(cap.isOpened()):
        current_frame += 1
        if current_frame % 10 == 0:
            print(current_frame, end = ", ")
      # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            img = load_image(frame)
            base_img = img.copy()

            # Face Detection
            detections = detect_face2(img = base_img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)
            if len(detections) == 0:
                pass
            else:
                bboxs = []
                face_count = 0
                for detection in detections:
                    x, y, w, h = detection["box"]
                    y_buffer = int(h * 0.5)
                    x_buffer = int(w * 0.5)
                    detected_face = base_img[max(0,int(y-y_buffer)):min(frame_height,int(y+h+y_buffer)), max(0,int(x-x_buffer)):min(frame_width,int(x+w+x_buffer))]

                        #Save the face
                    try:
                        detected_face = detected_face[:, :, ::-1]
                        new_im = Image.fromarray(detected_face)
                        new_im.save(output_dir + video.split(".")[0] + "_" + str(current_frame) + "_" + str(face_count) + ".jpg")
                    except:
                        print("Saving Error")
                    face_count += 1

      # Break the loop
        else: 
            break

    # When everything done, release the video capture object
    cap.release()
    cv2.destroyAllWindows()

['VID_20210312_144233.mp4', 'VID_20210326_145416.mp4', 'VID_20210326_145558.mp4', 'VID_20210326_145902.mp4', 'VID_20210326_150208.mp4', 'VID_20210326_150259.mp4', 'VID_20210326_150559.mp4', 'VID_20210326_150901.mp4', 'VID_20210326_151005.mp4', 'VID_20210326_151105.mp4', 'VID_20210326_151145.mp4', 'VID_20210326_151243.mp4', 'VID_20210326_151420.mp4', 'VID_20210326_151512.mp4']
10, 20, 30, 40, 10, 20, 30, 10, 20, 30, 40, 10, 20, 30, 40, 50, 60, 70, 80, 10, 10, 20, 30, 10, 20, 10, 20, 10, 10, 10, 20, 30, 

In [9]:
# Sort Faces 6373
import os
import shutil
new_dir = "dataset/new"
output_folder = "dataset"
persons = ['Cheng', 'JiaPeng', 'ZuoLin']
db_path = "db"

pics = os.listdir(new_dir)
for pic in pics:
    img_dir = new_dir + "/" + pic
    df = DeepFace.find(img_path = img_dir, db_path = db_path, enforce_detection=False)
    df.sort_values('VGG-Face_cosine', inplace=True, ascending=True)
    result = re.split(r' |/|\\', df['identity'].iloc[0])[1]
    print(result)
    output_dir = output_folder + "/" + result + "/" + pic
    shutil.move(img_dir, output_dir)

Analyzing:   0%|                                                                                 | 0/1 [00:00<?, ?it/s]

There are  10700  representations found in  representations_vgg_face.pkl


Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.71s/it]


find function lasts  9.16835069656372  seconds
ZuoLin


Analyzing:   0%|                                                                                 | 0/1 [00:00<?, ?it/s]

...

find function lasts  7.172994375228882  seconds
ZuoLin


Analyzing:   0%|                                                                                 | 0/1 [00:00<?, ?it/s]

There are  10700  representations found in  representations_vgg_face.pkl


Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.23s/it]

find function lasts  7.229997396469116  seconds
ZuoLin


In [ ]:
# Video Analysis

from deepface.commons.functions import initialize_detector, load_image, detect_face2
import numpy as np
import cv2
from PIL import Image, ImageDraw
current_frame = 1
video_path = 'dataset/demo3.mp4'
fps = 2
# detector_backend = 'ssd'
detector_backend = 'mtcnn'
grayscale = False
enforce_detection = False
return_region = False
db_folder = "db"

col_dict = {"ZuoLin" : "Green", "JiaPeng": "Red"}


initialize_detector(detector_backend = detector_backend)

cap = cv2.VideoCapture(video_path)

if (cap.isOpened()== False):
    print("Error opening video stream or file")

ret, frame = cap.read()
frame_width = frame.shape[1]
frame_height = frame.shape[0]
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))
out.write(frame)
# Read until video is completed
while(cap.isOpened()):
    current_frame += 1
    if current_frame % 10 == 0:
        print(current_frame, end = ", ")
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        img = load_image(frame)
        base_img = img.copy()
        
        # Add YOLO here
        bboxs = [[20, 20, 20, 20, 0.99, "Laptop"]]
        img = drawBBoxs(img, bboxs, col_dict)
        
        # Face Detection
        detections = detect_face2(img = base_img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)
        if len(detections) == 0:
            out.write(img)
        else:
            bboxs = []
            face_count = 0
            for detection in detections:
                x, y, w, h = detection["box"]
                detected_face = base_img[int(y):int(y+h), int(x):int(x+w)]
                face_detected = "Unknown"
                try:
                    new_im = Image.fromarray(detected_face)
                    new_im.save("current.jpg")
                    df = DeepFace.find(img_path = "current.jpg", db_path = "db", enforce_detection=False)
                    df.sort_values('VGG-Face_cosine', inplace=True, ascending=True)
                    face_detected = re.split(r' |/|\\', df['identity'].iloc[0])[1]
                except:
                    pass
                bboxs.append([x, y, w, h, detection['confidence'],face_detected])
                
                
#                     #Save the face
#                     try:
#                         detected_face = base_img[int(y):int(y+h), int(x):int(x+w)]
#                         detected_face = detected_face[:, :, ::-1]
#                         new_im = Image.fromarray(detected_face)
#                         new_im.save(db_folder + "/new/" + str(current_frame) + "_" + str(face_count) + ".jpg")
#                     except:
#                         pass
                face_count += 1
                    
            output_img = drawBBoxs(img, bboxs, col_dict)
            out.write(output_img)
  # Break the loop
    else: 
        break
    
# When everything done, release the video capture object
cap.release()
out.release()

cv2.destroyAllWindows()


In [13]:
# Face recognition
df = DeepFace.find(img_path = "dataset/img70.jpg", db_path = "db_face", enforce_detection=False)
print(df.head())
df.sort_values('VGG-Face_cosine', inplace=True, ascending=True)
result = re.split(r' |/|\\', df['identity'].iloc[0])[1]
print(result)

Analyzing:   0%|                                                                                 | 0/1 [00:00<?, ?it/s]

There are  370  representations found in  representations_vgg_face.pkl


Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.23s/it]

find function lasts  7.178085803985596  seconds
                                 identity  VGG-Face_cosine
0  db\ZuoLin/VID_20210312_144233_12_0.jpg         0.030339
1  db\ZuoLin/VID_20210312_144233_11_0.jpg         0.058416
2   db\ZuoLin/VID_20210312_144233_9_0.jpg         0.060761
3  db\ZuoLin/VID_20210312_144233_10_0.jpg         0.063052
4  db\ZuoLin/VID_20210312_144233_14_0.jpg         0.064872
ZuoLin


In [ ]:
# Video Analysis Old

from deepface.commons.functions import initialize_detector, load_image, detect_face2
import numpy as np
import cv2
from PIL import Image, ImageDraw
current_frame = 1
video_path = 'dataset/demo2.mp4'
# detector_backend = 'ssd'
detector_backend = 'mtcnn'
grayscale = False
enforce_detection = False
return_region = False
db_folder = "db"

initialize_detector(detector_backend = detector_backend)

cap = cv2.VideoCapture(video_path)

if (cap.isOpened()== False):
    print("Error opening video stream or file")

ret, frame = cap.read()
frame_width = frame.shape[1]
frame_height = frame.shape[0]
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
out.write(frame)
# Read until video is completed
while(cap.isOpened()):
    current_frame += 1
    if current_frame % 10 == 0:
        print(current_frame, end = ", ")
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        img = load_image(frame)
        
        
        if detector_backend == 'ssd':
            original_size = img.shape
            target_size = (300, 300)
            aspect_ratio_x = (original_size[1] / target_size[1])
            aspect_ratio_y = (original_size[0] / target_size[0])
            base_img = img.copy()
            df = detect_face2(img = img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)

            if df.shape[0] == 0: #No Face Detected
                out.write(img)

            else:

                df.sort_values('left', inplace=True)

                bboxs = []
                for index, instance in df.iterrows():
                    left = instance["left"]
                    right = instance["right"]
                    bottom = instance["bottom"]
                    top = instance["top"]
                    detected_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y), int(left*aspect_ratio_x):int(right*aspect_ratio_x)]
                    bboxs.append([int(left*aspect_ratio_x), int(top*aspect_ratio_y), int(right*aspect_ratio_x) - int(left*aspect_ratio_x), int(bottom*aspect_ratio_y) - int(top*aspect_ratio_y), instance['confidence']])

                output_img = drawBBoxs(base_img, bboxs)
                out.write(output_img)
                
        if detector_backend == 'mtcnn':
            base_img = img.copy()
            detections = detect_face2(img = img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)
            if len(detections) == 0:
                out.write(img)
            else:
                bboxs = []
                face_count = 0
                for detection in detections:
                    x, y, w, h = detection["box"]
                    bboxs.append([x, y, w, h, detection['confidence']])
#                     #Save the face
#                     try:
#                         detected_face = base_img[int(y):int(y+h), int(x):int(x+w)]
#                         detected_face = detected_face[:, :, ::-1]
#                         new_im = Image.fromarray(detected_face)
#                         new_im.save(db_folder + "/new/" + str(current_frame) + "_" + str(face_count) + ".jpg")
#                     except:
#                         pass
                    face_count += 1
                    
                output_img = drawBBoxs(base_img, bboxs)
                out.write(output_img)
  # Break the loop
    else: 
        break
    
# When everything done, release the video capture object
cap.release()
out.release()

cv2.destroyAllWindows()


In [ ]:
# Face recognition
df = DeepFace.find(img_path = "dataset/img71.jpg", db_path = "db")
print(df.head())

In [ ]:
from deepface.commons.functions import initialize_detector, preprocess_face, load_image, detect_face2
from PIL import Image, ImageDraw

detector_backend = 'mtcnn'
img_path = "dataset/img69.jpg"
grayscale = False
enforce_detection = False
return_region = False
initialize_detector(detector_backend)

img = load_image(img_path)
original_size = img.shape
detections = detect_face2(img = img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)
print(detections)

In [ ]:
from deepface.commons.functions import initialize_detector, preprocess_face, load_image, detect_face2
from PIL import Image, ImageDraw

detector_backend = 'ssd'
img_path = "dataset/img68.jpg"
target_size=(224, 224)
grayscale = False
enforce_detection = False
return_region = False

initialize_detector(detector_backend)

# detect_face ssd
img = load_image(img_path)
original_size = img.shape
target_size = (300, 300)
aspect_ratio_x = (original_size[1] / target_size[1])
aspect_ratio_y = (original_size[0] / target_size[0])

#Display original image
img2 = img[:, :, ::-1] #bgr to rgb
print("Original Image")
plt.imshow(img2)
plt.show()

base_img = img.copy()
df = detect_face2(img = img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection)
df.sort_values('left', inplace=True)
print(df)

bboxs = []

for index, instance in df.iterrows():
    left = instance["left"]
    right = instance["right"]
    bottom = instance["bottom"]
    top = instance["top"]
    detected_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y), int(left*aspect_ratio_x):int(right*aspect_ratio_x)]
    bboxs.append([int(left*aspect_ratio_x), int(top*aspect_ratio_y), int(right*aspect_ratio_x) - int(left*aspect_ratio_x), int(bottom*aspect_ratio_y) - int(top*aspect_ratio_y), instance['confidence']])
    
    img2 = detected_face[:, :, ::-1]
    plt.imshow(img2)
    plt.show()
    
output_img = base_img.copy()
output_img = output_img[:, :, ::-1]

output_img = Image.fromarray(np.uint8(output_img)).convert('RGB')
draw = ImageDraw.Draw(output_img)

for bbox in bboxs:
    x0 = bbox[0]
    x1 = bbox[0] + bbox[2]
    y0 = bbox[1]
    y1 = bbox[1] + bbox[3]
    draw.rectangle((x0, y0, x1, y1), outline="blue", width=5)
    draw.text((x0 + 10, y0 + 10), str(round(bbox[4],2)), fill="red")


plt.imshow(output_img)
plt.show()

In [ ]:
print(type(output_img))

In [ ]:
result  = DeepFace.verify("dataset/img1.jpg", "dataset/img2.jpg")
print(result)
print("Is verified: ", result["verified"])

In [ ]:
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace"
#           , "Dlib"
         ]
for model in models:
    result = DeepFace.verify("dataset/img1.jpg", "dataset/img2.jpg", model_name = model)
    print(model)
    print(result)
    print()

In [ ]:
# Face recognition
df = DeepFace.find(img_path = "dataset/img68.jpg", db_path = "db")
print(df.head())

In [ ]:
# Face Detectors
backends = ['opencv', 'ssd'
#             , 'dlib'
            , 'mtcnn']
for backend in backends:
   #face detection and alignment
    detected_face = DeepFace.detectFace("dataset/img61.jpg", detector_backend = backend)
    print(backend)
    plt.imshow(detected_face)
    plt.show()

In [ ]:
# DeepFace.stream(db_path = "db", model_name ='VGG-Face', distance_metric = 'cosine', enable_face_analysis = False, source = 'test.mkv', time_threshold = 5, frame_threshold = 5)